In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 들어가며

- 하위 디렉터리 알아내는 코드 참고자료 : https://wikidocs.net/39
- 본인의 파일위치에 맞게 변형해서 사용하면 된다.
- AI허브요약데이터셋(https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=582)에서 받은 데이터를 unzip해서 사용한다면 경로상 크게 문제가 없을 것이다.

## 데이터 가져오기

In [5]:
import glob
import json
import os
import pandas as pd

"""
함수 파라미터 소개
1.dirname : 본인이 파일을 가지고 있는 경로이며 train 혹은 val까지 연결된 파일경로를 넣어주어야한다.
  AI허브에요약 데이터셋을 UNZIP해서 바로 사용하려면 train, val가 아닌 한국어로 폴더명칭이 되어 있을 것이니 
  해당 함수에 입력한 경로를 그대로 넣으면 안된다.

2.tv_c : Val인지 train인지 체크 : train이면 /TL1.zip, val이면 /VL1.zip 입력
  추가적으로 경로가 꼬였다면 print()를 해서 확인해가면서 해당 코드를 변경하면 될 것이다. 

3.ps_c : /20per or /2~3sent 폴더 지정하기

4.sumnum : Summary1, Summary2, Summary3인지 지정한다. 
"""

def search(dirname, tv_c, ps_c, sumnum): 
    train_summary = []
    train_passage = []
    folder_list = []
    filenames = os.listdir(dirname) 
    for filename in filenames:
        full_filename = os.path.join(dirname, filename) # 여기에서는 10가지 종류의 폴더와 zip파일이 나올 것이다.
        if full_filename != dirname+ tv_c : # unzip을 하였기에 zip 파일을 걸러주는 조건문이다. 
          train_filenames = os.listdir(full_filename) # 여기에서는 20per, 2~3setn 폴더가 나올 것이다. 
          for train_filename in train_filenames: 
            train_full_filename = os.path.join(full_filename, train_filename) 
            if train_full_filename == full_filename + ps_c: # 여기에서는 20per, 2~3setn폴더중 어떤 곳으로 진입할지 지정한다.
              folder_list.append(train_full_filename+'/*.json') # .json파일을 모두 불러와 경로를 저장한다.
    # 해당 데이터에 맞게 summary와 passage를 불러오도록 구현된 코드이다. 
    for folder in folder_list: # 저장된 경로를 통해서 glob.glob에 넣어 파일을 읽어온다.
      for i in glob.glob(folder):
        f = open(i,'r')        
        js = json.loads(f.read())

        """
        1.summary1~3소개
        summary1 : 1문장 요약 / 20per 또는 2~3setn어디든 존재/ 20per에서 뽑아낸 1문장과 2~3setn에서 뽑아낸 한 문장이 존재한다는 뜻
        summary2 : 2~3문장 요약 (추출) / 2~3setn 폴더에만 존재한다.
        summary3 : 20% 문장 요약 (추출) / 20per 폴더에만 존재한다.

        2.고려사항 정리
        첫번째로 고려할 것은 train or val
        두번째로 고려할 것은 20per or 2~3setn 
        세번째로 고려할 것은 summary1~3
        """

        train_summary.append(js['Annotation'][sumnum])
        train_passage.append(js['Meta(Refine)']['passage'])
        f.close()
    return train_summary, train_passage



train은 약 20분 걸림

In [ ]:
# trian + 20per + Summary1 예시
train_summary, train_passage = search("/content/drive/MyDrive/인공지능/아이펠톤/PoC/data/train", '/TL1.zip', '/20per', 'summary1') 
print(train_summary[100])
print(train_passage[100])

In [ ]:
# val + 20per + Summary1예시
val_summary, val_passage = search("/content/drive/MyDrive/인공지능/아이펠톤/PoC/data/val", '/VL1.zip', '/20per', 'summary3') 
print(val_summary[100])
print(val_passage[100])

## train 데이터 DataFrame에 저장 및 csv 파일로 변환
- 예시코드일 뿐이니 저장 경로와 파일명은 조금 변형해야됨

In [ ]:
summary_name = "" #per_20, setn_2~3, onesetn
train_data = pd.DataFrame(zip(train_passage, train_summary), columns=['input','sentence_{}'.format()])
train_data.head()

In [ ]:
train_data.to_csv("train.csv", mode='w', index = False)
dff = pd.read_csv('/content/train.csv')
dff.head()

## val 데이터 DataFrame에 저장 및 csv 파일로 변환
- 예시코드일 뿐이니 저장 경로와 파일명은 조금 변형해야됨

In [ ]:
summary_name = "" #per_20, setn_2~3, onesetn
val_data = pd.DataFrame(zip(val_passage, val_summary), columns=['input','summary_{}'.format(summary_name)])
val_data.head()

In [ ]:
val_data.to_csv("val.csv", mode='w', index = False)
dff = pd.read_csv('/content/val.csv')
dff.head()